# Cleaning input raw data

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from dateutil import parser
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType, StringType

In [2]:
spark = SparkSession.builder.appName("Data Cleaning").master("local[*]").getOrCreate()
spark

In [3]:
input_df = spark.read.option("header", "true").option("inferSchema", "true").format("csv").load("../input/messy_ecommerce_1K.csv")
input_df.show(10)

+--------------------+----------------+----------+------------+-------------+--------------------+--------------------+-------------------+------------------+--------------+------------+---------------+--------------------------+-------------+--------------+----------+--------------------+----------------+-------------------+-------------+------------+------------------+------------+--------------+---------------+-----------------+----------------+--------------+--------------+--------------+----------------+----------------------+----------------------+-------------+--------------+------------+----------------+---------------+
|            order_id|      order_date|order_time|order_status|shipping_cost|         customer_id|      customer_email|customer_first_name|customer_last_name|customer_phone|customer_age|customer_gender|customer_registration_date|customer_city|customer_state|product_id|        product_name|product_category|product_subcategory|product_brand|product_cost|product_li

In [4]:
print(f"Number of rows: {input_df.count()}")
print(f"Number of columns: {len(input_df.columns)}")
print(f"Column names: {input_df.columns}")

Number of rows: 1000
Number of columns: 38
Column names: ['order_id', 'order_date', 'order_time', 'order_status', 'shipping_cost', 'customer_id', 'customer_email', 'customer_first_name', 'customer_last_name', 'customer_phone', 'customer_age', 'customer_gender', 'customer_registration_date', 'customer_city', 'customer_state', 'product_id', 'product_name', 'product_category', 'product_subcategory', 'product_brand', 'product_cost', 'product_list_price', 'warehouse_id', 'warehouse_city', 'warehouse_state', 'warehouse_country', 'quantity_ordered', 'payment_method', 'payment_status', 'order_returned', 'payment_refunded', 'shipping_address_line1', 'shipping_address_line2', 'shipping_city', 'shipping_state', 'shipping_zip', 'shipping_country', 'shipping_method']


### Cleaning order status

In [5]:
raw_order_status = input_df.select("order_status").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Raw order statuses list: {raw_order_status}")

print("Reading order status lookup table.")
order_status_lookup = spark.read.option("header", "true").option("inferSchema", True).format("csv").load("../lookup/order_status.csv")
# order_status_lookup.show()

clean_order_status_df = input_df.withColumn("order_status", trim(lower(col("order_status"))))

clean_order_status_df = clean_order_status_df.join(
    broadcast(order_status_lookup), 
    on=clean_order_status_df.order_status==order_status_lookup.raw_status,
    how="left").withColumn("order_status_cleaned", coalesce(col("clean_status"), lit("pending"))).drop("order_status", "raw_status", "clean_status")

cleaned_order_status = clean_order_status_df.select("order_status_cleaned").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Cleaned order statuses list: {cleaned_order_status}")

clean_order_status_df.show()



Raw order statuses list: ['pending|pending_alt', 'cancelled   ', 'pen3ing', 'shipped', '  cancelled  ', '  pending  ', '  shipped  ', 'cancelled', 'shipped|shipped_alt', 'delxvered', 'pen@ing', 'can3elled', 'cancelled|cancelled_alt', 'dexivered', 'pend1ng', 'delivered', 'pending', 'delivered|delivered_alt', 'pending   ', 'PENDING', None]
Reading order status lookup table.
Cleaned order statuses list: ['success', 'failed', 'pending']
+--------------------+----------------+----------+-------------+--------------------+--------------------+-------------------+------------------+--------------+------------+---------------+--------------------------+-------------+--------------+----------+--------------------+----------------+-------------------+-------------+------------+------------------+------------+--------------+---------------+-----------------+----------------+--------------+--------------+--------------+----------------+----------------------+----------------------+-------------+--

### Clean dates - transforming dates data to standard form

In [6]:
def transform_date_udf(val):
    if val is None:
        return None
    return parser.parse(val).date()

transform_date_udf = udf(transform_date_udf, DateType())

clean_dates_df = clean_order_status_df.withColumn(
    "order_date_cleaned",
    transform_date_udf(col("order_date"))
).withColumn(
    "customer_registration_date_cleaned", 
    transform_date_udf(col("customer_registration_date"))
    ).drop("order_date", "customer_registration_date")
clean_dates_df.show()

+--------------------+----------+-------------+--------------------+--------------------+-------------------+------------------+--------------+------------+---------------+-------------+--------------+----------+--------------------+----------------+-------------------+-------------+------------+------------------+------------+--------------+---------------+-----------------+----------------+--------------+--------------+--------------+----------------+----------------------+----------------------+-------------+--------------+------------+----------------+---------------+--------------------+------------------+----------------------------------+
|            order_id|order_time|shipping_cost|         customer_id|      customer_email|customer_first_name|customer_last_name|customer_phone|customer_age|customer_gender|customer_city|customer_state|product_id|        product_name|product_category|product_subcategory|product_brand|product_cost|product_list_price|warehouse_id|warehouse_city|wa

### Transform phone number to standard format

In [7]:
cleaned_phone_number_df = clean_dates_df.withColumn(
    "customer_phone_cleaned", regexp_replace(col("customer_phone") , r"[^0-9]", "")
).withColumn(
    "customer_phone_cleaned", when(length(col("customer_phone_cleaned"))>10, col("customer_phone_cleaned").substr(2, 10)).otherwise(col("customer_phone_cleaned"))
).withColumn(
    "customer_phone_cleaned", concat(lit("("), col("customer_phone_cleaned").substr(1, 3), lit(")"), lit(" "), col("customer_phone_cleaned").substr(4, 3), lit("-"), col("customer_phone_cleaned").substr(7, 4))
).drop("customer_phone")
cleaned_phone_number_df.show()

+--------------------+----------+-------------+--------------------+--------------------+-------------------+------------------+------------+---------------+-------------+--------------+----------+--------------------+----------------+-------------------+-------------+------------+------------------+------------+--------------+---------------+-----------------+----------------+--------------+--------------+--------------+----------------+----------------------+----------------------+-------------+--------------+------------+----------------+---------------+--------------------+------------------+----------------------------------+----------------------+
|            order_id|order_time|shipping_cost|         customer_id|      customer_email|customer_first_name|customer_last_name|customer_age|customer_gender|customer_city|customer_state|product_id|        product_name|product_category|product_subcategory|product_brand|product_cost|product_list_price|warehouse_id|warehouse_city|warehouse

### Clean payment status - map each payment status to 'success', 'failed' or 'pending'

In [8]:
raw_payment_status = cleaned_phone_number_df.select("payment_status").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Raw payment statuses list: {raw_order_status}")

payment_status_lookup = spark.read.option("header", "true").option("inferSchema", "true").format("csv").load("../lookup/payment_status.csv")
payment_status_lookup.show(truncate=False)

cleaned_phone_number_df = cleaned_phone_number_df.withColumn("payment_status", trim(col("payment_status")))

cleaned_payment_status_df = cleaned_phone_number_df.join(
    broadcast(payment_status_lookup), 
    on=cleaned_phone_number_df.payment_status==payment_status_lookup.raw_value,
    how="left"
    ).withColumnRenamed(
        "standard_value", "payment_status_cleaned"
        ).drop("raw_value", "payment_status")

cleaned_payment_status_df.show(truncate=False)

cleaned_payment_status = cleaned_payment_status_df.select("payment_status_cleaned").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Cleaned payment statuses list: {cleaned_payment_status}")



Raw payment statuses list: ['pending|pending_alt', 'cancelled   ', 'pen3ing', 'shipped', '  cancelled  ', '  pending  ', '  shipped  ', 'cancelled', 'shipped|shipped_alt', 'delxvered', 'pen@ing', 'can3elled', 'cancelled|cancelled_alt', 'dexivered', 'pend1ng', 'delivered', 'pending', 'delivered|delivered_alt', 'pending   ', 'PENDING', None]
+-------------------+--------------+
|raw_value          |standard_value|
+-------------------+--------------+
|pending|pending_alt|pending       |
|pending            |pending       |
|pen@ing            |pending       |
|penying            |pending       |
|pen#ing            |pending       |
|pendyng            |pending       |
|pendxng            |pending       |
|PENDING            |pending       |
|success            |success       |
|SUCCESS            |success       |
|success|success_alt|success       |
|succ@ss            |success       |
|succexs            |success       |
|failed             |failed        |
|fail1d             |failed  

### Clean payment method

In [9]:
raw_payment_methods = cleaned_payment_status_df.select("payment_method").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Raw payment methods list: {raw_payment_methods}")

cleaned_payment_method_df = cleaned_payment_status_df.withColumn("payment_method", trim(col("payment_method")))

payment_methods_lookup = spark.read.option("inferSchema", True).option("header", True).csv("../lookup/payment_method.csv")
payment_methods_lookup.show()

cleaned_payment_method_df = cleaned_payment_method_df.join(
    broadcast(payment_methods_lookup),
    on=cleaned_payment_method_df.payment_method==payment_methods_lookup.raw_value,
    how="left"
).withColumnRenamed("standard_value", "payment_method_cleaned").drop("raw_value").drop("payment_method")

cleaned_payment_methods = cleaned_payment_method_df.select(col("payment_method_cleaned")).distinct().rdd.flatMap(lambda x: x).collect()
print(f"Cleaned payment methods list: {cleaned_payment_methods}")

cleaned_payment_method_df.show(10, truncate=False)

Raw payment methods list: ['  debit_card  ', 'appleypay', 'google_pay', 'paypal   ', 'google_pay   ', 'paypal', 'credit_card   ', 'paypal|paypal_alt', 'deyit_card', 'goo2le_pay', 'pazpal', 'crexit_card', 'pxypal', '  google_pay  ', 'ap3le_pay', 'apple_pay', 'credit_card', 'debit_card', '  credit_card  ', 'credit_card|credit_card_alt', 'apple_pay   ', 'GOOGLE_PAY', 'gooyle_pay', 'apple_pay|apple_pay_alt', None]
+--------------------+--------------+
|           raw_value|standard_value|
+--------------------+--------------+
|          debit_card|    debit_card|
|debit_card|debit_...|    debit_card|
|          debit_#ard|    debit_card|
|google_pay|google...|    google_pay|
|          go1gle_pay|    google_pay|
|          googl@_pay|    google_pay|
|          googly_pay|    google_pay|
|          GOOGLE_PAY|    google_pay|
|              paypal|        paypal|
|   paypal|paypal_alt|        paypal|
|           APPLE_PAY|     apple_pay|
|           apple_pay|     apple_pay|
|apple_pay|apple

### Clean Age to standard format

In [10]:
cleaned_age_df = (cleaned_payment_method_df
                  .withColumn("customer_age_cleaned", regexp_extract(col("customer_age"), r"(\d+)", 1).cast("int"))
                  .withColumn("customer_age_cleaned", when((col("customer_age_cleaned")>99) | (col("customer_age_cleaned")<15), None).otherwise(col("customer_age_cleaned")))
                  .drop("customer_age")
                  )

cleaned_age_df.select("customer_age_cleaned").show()

+--------------------+
|customer_age_cleaned|
+--------------------+
|                  52|
|                  66|
|                  38|
|                  51|
|                  68|
|                  75|
|                  38|
|                  48|
|                  72|
|                  27|
|                  80|
|                  42|
|                  44|
|                  67|
|                  69|
|                  54|
|                  80|
|                  64|
|                  37|
|                  36|
+--------------------+
only showing top 20 rows



In [13]:
raw_genders = cleaned_age_df.select("customer_gender").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Raw genders list: {raw_genders}")

cleaned_gender_df = (cleaned_age_df
                     .withColumn("customer_gender_cleaned", trim(col("customer_gender")))
                     .withColumn("customer_gender_cleaned", lower(col("customer_gender_cleaned")))
                     .withColumn("customer_gender_cleaned", when(col("customer_gender_cleaned").contains("f"), "female")
                                 .otherwise(when(col("customer_gender_cleaned").contains("m"), "male")
                                            .otherwise("other")))
                     .drop("customer_gender")
                     )

cleaned_genders = cleaned_gender_df.select("customer_gender_cleaned").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Cleaned genders list: {cleaned_genders}")

cleaned_gender_df.show(truncate=False)

Raw genders list: ['  Other  ', 'Other|Other_alt', 'Oyher', 'F', '  F  ', 'Othyr', 'm', 'F|F_alt', 'f', 'M', 'other', 'Other', 'O2her', 'F   ', 'Other   ', '  M  ', 'Otxer', 'M|M_alt', None]
Cleaned genders list: ['female', 'other', 'male']
+------------------------------------+----------+-------------+------------------------------------+---------------------------+-------------------+------------------+-------------+--------------+----------+--------------------------------+----------------+-------------------+-------------+------------+------------------+------------+--------------+---------------+-----------------+----------------+--------------+----------------+----------------------+----------------------+-------------+--------------+------------+----------------+---------------+--------------------+------------------+----------------------------------+----------------------+----------------------+----------------------+--------------------+-----------------------+
|order_id     

### Clean payment refunded column to standard values

In [17]:
raw_payment_refund_status = cleaned_gender_df.select("payment_refunded").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Raw payment refund status list: {raw_payment_refund_status}")

print("Reading payment refund status lookup table.")
payment_refund_lookup = spark.read.option("header", True).option("inferSchema", True).format("csv").load("../lookup/payment_refund.csv")

print("Cleaned payment refund status")
cleaned_payment_refunded_df = (cleaned_gender_df
                               .join(
                                   payment_refund_lookup, 
                                   on=cleaned_gender_df.payment_refunded==payment_refund_lookup.raw_value, 
                                   how="left")
                               .withColumnRenamed("standard_value", "payment_refund_status_cleaned")
                               .drop("raw_value")
                               )

cleaned_payment_refund_status = cleaned_payment_refunded_df.select("payment_refund_status_cleaned").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Raw payment refund status list: {cleaned_payment_refund_status}")

cleaned_payment_refunded_df.show()


Raw payment refund status list: ['pending|pending_alt', '1|1_alt', 'FALSE', 'falze', '  0  ', 'yes|yes_alt', '0', 'true|true_alt', 'TRUE', '  yes  ', 'falye', 'yes   ', '  false  ', 'fxlse', 'false|false_alt', 'false', 'pending', 'no|no_alt', '1', 'no', 'yes', 'true   ', 'NO', '  1  ', '  true  ', '0|0_alt', 'true', 'tr1e', '1   ', None]
Reading payment refund status lookup table.
Cleaned payment refund status
Raw payment refund status list: ['false', 'pending', 'true', None]
+--------------------+----------+-------------+--------------------+--------------------+-------------------+------------------+-------------+--------------+----------+--------------------+----------------+-------------------+-------------+------------+------------------+------------+--------------+---------------+-----------------+----------------+--------------+----------------+----------------------+----------------------+-------------+--------------+------------+----------------+---------------+---------------

### Clean order returned status to standard values

In [ ]:
raw_order_returned_status = cleaned_payment_refunded_df.select("order_returned").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Raw order returned status list: {raw_order_returned_status}")

print("Reading order returned status lookup table.")
order_returned_lookup = spark.read.option("header", True).option("inferSchema", True).format("csv").load("../lookup/order_returned.csv")

print("Cleaned order returned status")
clean_order_returned_df = (cleaned_payment_refunded_df
                           .join(
                               order_returned_lookup,
                               on=cleaned_payment_refunded_df.order_returned==order_returned_lookup.raw_value,
                               how="left"
                               )
                           .withColumnRenamed("standard_value", "order_returned_status_cleaned")
                           .drop("raw_value")
                           )

cleaned_order_returned_status = clean_order_returned_df.select("order_returned_status_cleaned").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Cleaned order returned status list: {cleaned_order_returned_status}")

clean_order_returned_df.show()


Raw order returned status list: ['pending|pending_alt', '1|1_alt', 'FALSE', 'tyue', '0   ', 'yes|yes_alt', '0', 'true|true_alt', '  pending  ', '  no  ', 'TRUE', 'false   ', 'YES', 'pend2ng', 'fa1se', 'yes   ', '  false  ', 'false|false_alt', 'false', 'pending', 'no|no_alt', 'no   ', '1', 'no', 'fal1e', 'yes', 'true   ', 'pending   ', 'PENDING', '  1  ', '  true  ', '0|0_alt', 'true', '1   ', None]
Reading order returned status lookup table.
Cleaned order returned status
+--------------------+----------+-------------+--------------------+--------------------+-------------------+------------------+-------------+--------------+----------+--------------------+----------------+-------------------+-------------+------------+------------------+------------+--------------+---------------+-----------------+----------------+--------------+----------------+----------------------+----------------------+-------------+--------------+------------+----------------+---------------+--------------------

: 